In [12]:
import pandas as pd
import numpy as np
import os
from scipy.stats import fisher_exact, ttest_rel, ttest_ind, binom_test
import json

# Load data

In [2]:
import pickle
infile = open("interaction_data.pkl",'rb')
d = pickle.load(infile)
infile.close()

infile = open("phase3_selections.pkl",'rb')
selected_item_interactions = pickle.load(infile)
infile.close()

infile = open("df_completed_participation.pkl",'rb')
df_completed_participation = pickle.load(infile)
infile.close()

# Would the results be affected if we deliberately select the wrong design?
## All selections, deliberately selecting disadvantaged / advantaged slot for one method

In [3]:
selected_item_interactions["selected_algorithm"] = "GAMMA"
selected_item_interactions.loc[selected_item_interactions.variant == selected_item_interactions.DELTA, "selected_algorithm"] = "DELTA"

In [4]:
selected_with_alg = selected_item_interactions.groupby(["participation","iteration","result_layout","selected_algorithm","variant"])[["DELTA"]].count()
selected_with_alg.columns = ["selected_count"]
selected_with_alg = selected_with_alg.reset_index([3,4])
selected_with_alg.head()

v0 = selected_with_alg.loc[selected_with_alg.selected_algorithm=="DELTA"]
v1 = selected_with_alg.loc[selected_with_alg.selected_algorithm=="GAMMA"]
variant_joined = v0.join(v1, how="outer", rsuffix="_v1")
variant_joined["selected_algorithm"] = "DELTA"
variant_joined["selected_algorithm_v1"] = "GAMMA"
variant_joined.fillna(0, inplace=True)
variant_joined = variant_joined.reset_index()
variant_joined

,participation,iteration,result_layout,selected_algorithm,variant,selected_count,selected_algorithm_v1,variant_v1,selected_count_v1
0,36,1.0,row-single-scrollable,DELTA,0.0,4.0,GAMMA,1.0,2.0
1,36,2.0,columns,DELTA,0.0,0.0,GAMMA,0.0,5.0
2,36,3.0,max-columns,DELTA,1.0,1.0,GAMMA,0.0,1.0
3,36,4.0,rows,DELTA,0.0,4.0,GAMMA,0.0,0.0
4,36,5.0,row-single-scrollable,DELTA,1.0,1.0,GAMMA,0.0,1.0
...,...,...,...,...,...,...,...,...,...
1044,261,4.0,max-columns,DELTA,0.0,3.0,GAMMA,1.0,3.0
1045,261,5.0,rows,DELTA,0.0,1.0,GAMMA,1.0,2.0
1046,261,6.0,row-single-scrollable,DELTA,1.0,2.0,GAMMA,0.0,4.0
1047,261,7.0,columns,DELTA,1.0,3.0,GAMMA,0.0,2.0


In [7]:
dx = variant_joined["selected_count"]
dy = variant_joined["selected_count_v1"]
print(dx.mean(), dy.mean())
print(ttest_rel(dx,dy))

3.724499523355577 2.742612011439466
Ttest_relResult(statistic=11.768579083194775, pvalue=4.0561124250320255e-30)


- Delta significantly outperformed gamma with p close to 0

In [8]:
for i in variant_joined.variant.unique():
    dx = variant_joined.loc[variant_joined.variant == i,"selected_count"]
    dy = variant_joined.loc[variant_joined.variant == i,"selected_count_v1"]
    print("variant", i)
    print(dx.mean(), dy.mean())
    print(ttest_rel(dx,dy))
    print("")

variant 0.0
3.7795698924731185 2.467741935483871
Ttest_relResult(statistic=11.124798770556337, pvalue=4.238618584024765e-26)

variant 1.0
3.6619144602851326 3.054989816700611
Ttest_relResult(statistic=5.261644045051465, pvalue=2.138461991416444e-07)



In [9]:
for i in variant_joined.variant.unique():
    for j in variant_joined.result_layout.unique():
        dx = variant_joined.loc[((variant_joined.variant == i)&(variant_joined.result_layout==j)),"selected_count"]
        dy = variant_joined.loc[((variant_joined.variant == i)&(variant_joined.result_layout==j)),"selected_count_v1"]
        print("variant", i,j)
        print(dx.mean(), dy.mean())
        print(ttest_rel(dx,dy))
        print("")

variant 0.0 row-single-scrollable
3.206896551724138 2.2988505747126435
Ttest_relResult(statistic=3.2691083511886783, pvalue=0.0015522311675777935)

variant 0.0 columns
3.63265306122449 2.5510204081632653
Ttest_relResult(statistic=3.8473210962379505, pvalue=0.0002138042916200897)

variant 0.0 max-columns
4.089887640449438 2.5280898876404496
Ttest_relResult(statistic=4.537411622639667, pvalue=1.7935587853285236e-05)

variant 0.0 rows
3.956989247311828 2.6774193548387095
Ttest_relResult(statistic=4.639792852673591, pvalue=1.1522773256267972e-05)

variant 0.0 column-single
3.784313725490196 2.588235294117647
Ttest_relResult(statistic=4.213673365089623, pvalue=5.4738500370854116e-05)

variant 0.0 row-single
4.0 2.1235955056179776
Ttest_relResult(statistic=7.215596366342409, pvalue=1.7959817891912334e-10)

variant 1.0 row-single-scrollable
3.546875 3.0
Ttest_relResult(statistic=1.9477392655515526, pvalue=0.05590532427626266)

variant 1.0 columns
3.7093023255813953 2.941860465116279
Ttest_rel

- considering only the data, where Delta was in disadvantaged slot and rows:double layout, we would be unable to confirm its supremacy over gamma

# All selections, bootstraping analysis: 
## how many participants to get significant results
### within subject design (all participants has both ordering of methods)

In [19]:
rt1 = d.drop_duplicates(["participation","iteration","DELTA"]).set_index(["participation","iteration","DELTA"])[["result_layout"]]
sii = rt1.join(selected_item_interactions.set_index(["participation","iteration","DELTA"]), how="outer", rsuffix="_r")
sii = sii.reset_index().fillna(0)
sii.shape

(6828, 13)

In [70]:
contingency_table = pd.crosstab(sii.result_layout,sii.selected_algorithm)
participants_volume = 20
np.random.seed(42)
res = {}
for i in contingency_table.index.unique():
    resAgg = []
    resAggSign = []
    participantsPool = sii.loc[sii.result_layout == i,"participation"].unique()
    for j in range(1000):                
        participants = np.random.choice(participantsPool, participants_volume, True)
        results = sii.loc[sii.result_layout == i].set_index("participation").loc[participants]
        groups = results.groupby("selected_algorithm")[["movieID"]].count()
        resAgg.append(groups.loc["DELTA","movieID"] >= groups.loc["GAMMA","movieID"])
        
        succ = groups.loc["DELTA","movieID"]
        fail = groups.loc["GAMMA","movieID"]
        bt = binom_test(succ, succ+fail,p=0.5,alternative='greater')
        resAggSign.append(bt)        
        
    print(i, np.mean(resAgg), np.sum(np.array(resAggSign)<=0.05))
    res[i] = [np.mean(resAgg), np.sum(np.array(resAggSign)<=0.05)]
resultsWS = pd.DataFrame(res).T

column-single 0.979 670
columns 0.989 727
max-columns 0.998 932
row-single 1.0 924
row-single-scrollable 0.989 582
rows 0.95 454


### between-subject design: each participant is assigned to only one ordering

In [71]:
# using only the first occurence per user
sii4 = sii.loc[sii.iteration <=4]

In [72]:
np.random.seed(42)
res = {}
for i in contingency_table.index.unique():
    resAgg = []
    resAggSign = []    
    participantsPool = sii4.loc[sii4.result_layout == i,["participation","DELTA"]].drop_duplicates()
    for j in range(1000):                
        participantsIDs = np.random.choice(participantsPool.index, participants_volume*2, True)

        results = sii4.loc[sii4.result_layout == i].set_index(["participation","DELTA"]).loc[participantsPool.loc[participantsIDs].values.tolist()]
        groups = results.groupby("selected_algorithm")[["movieID"]].count()
        resAgg.append(groups.loc["DELTA"] >= groups.loc["GAMMA"])
        
        succ = groups.loc["DELTA","movieID"]
        fail = groups.loc["GAMMA","movieID"]
        bt = binom_test(succ, succ+fail,p=0.5,alternative='greater')
        resAggSign.append(bt)  
        
    print(i, np.mean(resAgg), np.sum(np.array(resAggSign)<=0.05))
    res[i] = [np.mean(resAgg), np.sum(np.array(resAggSign)<=0.05)]
resultsBS = pd.DataFrame(res).T

column-single 0.988 862
columns 0.971 525
max-columns 0.997 913
row-single 0.985 772
row-single-scrollable 0.975 507
rows 0.858 330


In [73]:
resultsWS - resultsBS

,0,1
column-single,-0.009,-192.0
columns,0.018,202.0
max-columns,0.001,19.0
row-single,0.015,152.0
row-single-scrollable,0.014,75.0
rows,0.092,124.0


In [74]:
totalVol = 6000
contingencyTable = [[resultsWS.sum()[1], totalVol - resultsWS.sum()[1]],[resultsBS.sum()[1], totalVol - resultsBS.sum()[1]]]
fisher_exact(contingencyTable)

(1.3408938505889165, 9.986574183045696e-14)

In [75]:
contingencyTable = [[resultsWS.sum()[0]*1000, totalVol - resultsWS.sum()[0]*1000],[resultsBS.sum()[0]*1000, totalVol - resultsBS.sum()[0]*1000]]
fisher_exact(contingencyTable)

(2.433342146288986, 6.180018853346323e-14)

In [76]:
for i in resultsWS.index.unique():
    if i != "rows":
        contingencyTable = [[resultsWS.loc[i,1], 1000-resultsWS.loc[i,1]],[resultsWS.loc["rows",1], 1000-resultsWS.loc["rows",1]]]
        print(i,fisher_exact(contingencyTable))

column-single (2.4417300760913094, 2.2348199224345637e-22)
columns (3.202643171806167, 1.2228111145914884e-35)
max-columns (16.483285825343355, 3.865802896207846e-130)
row-single (14.621609088801298, 4.5442513720596355e-124)
row-single-scrollable (1.6744936028497355, 1.2645251242954949e-08)


In [77]:
resultsWS

,0,1
column-single,0.979,670.0
columns,0.989,727.0
max-columns,0.998,932.0
row-single,1.000,924.0
row-single-scrollable,0.989,582.0
rows,0.950,454.0


### Results
- for both the volume of concordant results and the volume of stat. sign. concordant results, we observed that within subject design gives more clear results than between-subject, even if the volume of its participants doubles
- the rows:double has smallest chance to generate stat. sign results, e.g. below 50% for 20 participants

# First selections, bootstraping analysis: 
## how many participants to get significant results

In [79]:
selected_item_interactions = selected_item_interactions.sort_values(["participation","iteration","result_layout","time"])
first_selections = selected_item_interactions.drop_duplicates(["participation","iteration"])

first_selections["selected_algorithm"] = "GAMMA"
first_selections.loc[first_selections.variant == first_selections.DELTA, "selected_algorithm"] = "DELTA"
first_selections.head()

<ipython-input-79-09193f7f5e0a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_selections["selected_algorithm"] = "GAMMA"
C:\Users\lpesk\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,participation,interaction_type,time,data,iteration,result_layout,GAMMA,DELTA,variant,layout_high_level,movieID,selected_algorithm
id,,,,,,,,,,,,
3271,36,selected-item,2023-01-15 23:15:59.742434,"{""selected_item"": {""genres"": [""Adventure"", ""Fa...",1.0,row-single-scrollable,1.0,0.0,0,rows,106489,DELTA
3298,36,selected-item,2023-01-15 23:16:38.416542,"{""selected_item"": {""genres"": [""Adventure"", ""Fa...",2.0,columns,0.0,1.0,0,cols,118696,GAMMA
3314,36,selected-item,2023-01-15 23:17:09.189259,"{""selected_item"": {""genres"": [""Drama"", ""Thrill...",3.0,max-columns,0.0,1.0,1,cols,116797,DELTA
3325,36,selected-item,2023-01-15 23:17:30.629024,"{""selected_item"": {""genres"": [""Action"", ""Myste...",4.0,rows,1.0,0.0,0,rows,5418,DELTA
3345,36,selected-item,2023-01-15 23:18:10.680798,"{""selected_item"": {""genres"": [""Action"", ""Crime...",5.0,row-single-scrollable,0.0,1.0,0,rows,168248,GAMMA


In [103]:
contingency_table = pd.crosstab(first_selections.result_layout,first_selections.selected_algorithm)
contingency_table

selected_algorithm,DELTA,GAMMA
result_layout,,
column-single,101,94
columns,101,83
max-columns,92,78
row-single,90,80
row-single-scrollable,82,69
rows,90,86


In [107]:
succ = contingency_table["DELTA"].sum()
fail = contingency_table["GAMMA"].sum()
print(binom_test(succ, succ+fail,p=0.5,alternative='greater'))

0.02220170664276684


In [106]:
succ,fail

(556, 490)

In [82]:
deltaAdvantage = first_selections.loc[first_selections.DELTA==0]
gammaAdvantage = first_selections.loc[first_selections.DELTA==1]
deltaAdvantage.shape, gammaAdvantage.shape

((523, 12), (523, 12))

In [83]:
contingency_table = pd.crosstab(deltaAdvantage.result_layout,deltaAdvantage.selected_algorithm)
contingency_table

selected_algorithm,DELTA,GAMMA
result_layout,,
column-single,77,21
columns,75,16
max-columns,72,13
row-single,75,9
row-single-scrollable,68,9
rows,81,7


In [84]:
contingency_table = pd.crosstab(gammaAdvantage.result_layout,gammaAdvantage.selected_algorithm)
contingency_table

selected_algorithm,DELTA,GAMMA
result_layout,,
column-single,24,73
columns,26,67
max-columns,20,65
row-single,15,71
row-single-scrollable,14,60
rows,9,79


### Within subject

In [122]:
import random
participants_volume = 200
res = {}
np.random.seed(42)
for i in contingency_table.index.unique():
    resAgg = []
    resAggSign = []    
    participantsPool = first_selections.loc[first_selections.result_layout == i,"participation"].unique()
    for j in range(1000):                
        participants = np.random.choice(participantsPool, participants_volume, True)
        results = first_selections.loc[first_selections.result_layout == i].set_index("participation").loc[participants]
        groups = results.groupby("selected_algorithm")[["movieID"]].count()
        resAgg.append(groups.loc["DELTA"] >= groups.loc["GAMMA"])
        
        succ = groups.loc["DELTA","movieID"]
        fail = groups.loc["GAMMA","movieID"]
        bt = binom_test(succ, succ+fail,p=0.5,alternative='greater')
        resAggSign.append(bt)  
        
    print(i, np.mean(resAgg), np.sum(np.array(resAggSign)<=0.05))
    res[i] = [np.mean(resAgg), np.sum(np.array(resAggSign)<=0.05)]
resultsWS = pd.DataFrame(res).T    

column-single 0.812 104
columns 0.991 620
max-columns 0.981 462
row-single 0.957 214
row-single-scrollable 0.999 492
rows 0.803 6


### Between subject

In [123]:
# using only the first occurence per user
fi4 = first_selections.loc[first_selections.iteration <=4]

In [124]:
np.random.seed(42)
res = {}
for i in contingency_table.index.unique():
    resAgg = []
    resAggSign = []    
    participantsPool = fi4.loc[fi4.result_layout == i,["participation","DELTA"]].drop_duplicates()
    for j in range(1000):                
        participantsIDs = np.random.choice(participantsPool.index, participants_volume*2, True)

        results = fi4.loc[fi4.result_layout == i].set_index(["participation","DELTA"]).loc[participantsPool.loc[participantsIDs].values.tolist()]
        groups = results.groupby("selected_algorithm")[["movieID"]].count()
        resAgg.append(groups.loc["DELTA"] >= groups.loc["GAMMA"])
        
        succ = groups.loc["DELTA","movieID"]
        fail = groups.loc["GAMMA","movieID"]
        bt = binom_test(succ, succ+fail,p=0.5,alternative='greater')
        resAggSign.append(bt)  
        
    print(i, np.mean(resAgg), np.sum(np.array(resAggSign)<=0.05))
    res[i] = [np.mean(resAgg), np.sum(np.array(resAggSign)<=0.05)]
resultsBS = pd.DataFrame(res).T

column-single 0.987 695
columns 0.968 606
max-columns 1.0 979
row-single 0.98 700
row-single-scrollable 0.509 41
rows 0.003 0


## First three selected items

In [97]:
selected_item_interactions["selectionOrder"] = selected_item_interactions.groupby(["participation","iteration"])["time"].rank(method="dense", ascending=True)

In [98]:
firstThreeSelected = selected_item_interactions.loc[selected_item_interactions.selectionOrder <=3]
f3i = rt1.join(firstThreeSelected.set_index(["participation","iteration","DELTA"]), how="outer", rsuffix="_r")
f3i = f3i.reset_index().fillna(0)
f3i.shape

(3013, 14)

In [109]:
contingency_table = pd.crosstab(firstThreeSelected.result_layout,firstThreeSelected.selected_algorithm)
contingency_table

selected_algorithm,DELTA,GAMMA
result_layout,,
column-single,312,241
columns,295,231
max-columns,284,203
row-single,285,192
row-single-scrollable,237,177
rows,268,242


In [111]:
succ = contingency_table["DELTA"].sum()
fail = contingency_table["GAMMA"].sum()
print(binom_test(succ, succ+fail,p=0.5,alternative='greater'))

2.1812724736309596e-13


In [112]:
succ,fail

(1681, 1286)

### Within subject

In [119]:
participants_volume = 50
np.random.seed(42)
res = {}
for i in contingency_table.index.unique():
    resAgg = []
    resAggSign = []
    participantsPool = f3i.loc[f3i.result_layout == i,"participation"].unique()
    for j in range(1000):                
        participants = np.random.choice(participantsPool, participants_volume, True)
        results = f3i.loc[f3i.result_layout == i].set_index("participation").loc[participants]
        groups = results.groupby("selected_algorithm")[["movieID"]].count()
        resAgg.append(groups.loc["DELTA","movieID"] >= groups.loc["GAMMA","movieID"])
        
        succ = groups.loc["DELTA","movieID"]
        fail = groups.loc["GAMMA","movieID"]
        bt = binom_test(succ, succ+fail,p=0.5,alternative='greater')
        resAggSign.append(bt)        
        
    print(i, np.mean(resAgg), np.sum(np.array(resAggSign)<=0.05))
    res[i] = [np.mean(resAgg), np.sum(np.array(resAggSign)<=0.05)]
resultsWS = pd.DataFrame(res).T

column-single 0.997 684
columns 0.957 597
max-columns 1.0 844
row-single 0.997 907
row-single-scrollable 0.99 747
rows 0.823 229


### Between subject

In [120]:
# using only the first occurence per user
f3i4 = f3i.loc[f3i.iteration <=4]

In [121]:
np.random.seed(42)
res = {}
for i in contingency_table.index.unique():
    resAgg = []
    resAggSign = []    
    participantsPool = f3i4.loc[f3i4.result_layout == i,["participation","DELTA"]].drop_duplicates()
    for j in range(1000):                
        participantsIDs = np.random.choice(participantsPool.index, participants_volume*2, True)

        results = f3i4.loc[f3i4.result_layout == i].set_index(["participation","DELTA"]).loc[participantsPool.loc[participantsIDs].values.tolist()]
        groups = results.groupby("selected_algorithm")[["movieID"]].count()
        resAgg.append(groups.loc["DELTA"] >= groups.loc["GAMMA"])
        
        succ = groups.loc["DELTA","movieID"]
        fail = groups.loc["GAMMA","movieID"]
        bt = binom_test(succ, succ+fail,p=0.5,alternative='greater')
        resAggSign.append(bt)  
        
    print(i, np.mean(resAgg), np.sum(np.array(resAggSign)<=0.05))
    res[i] = [np.mean(resAgg), np.sum(np.array(resAggSign)<=0.05)]
resultsBS = pd.DataFrame(res).T

column-single 0.937 588
columns 0.99 840
max-columns 1.0 987
row-single 0.996 928
row-single-scrollable 0.834 432
rows 0.273 33
